# ETL: Limpieza de Datos de Viviendas en Barcelona

## Importamos las librerías necesarias


In [15]:
import pandas as pd
import numpy as np
import re

print("✅ Todas las librerías están importadas correctamente")


✅ Todas las librerías están importadas correctamente


## EXTRACT: Cargar los datos


In [16]:
# Cargar el CSV
df_raw = pd.read_csv("../data/housing-barcelona.csv")

print("✅ El dataframe se ha creado correctamente")
print(f"Shape: {df_raw.shape}")
print(f"\nColumnas: {list(df_raw.columns)}")
print(f"\nPrimeras filas:")
df_raw.head()


✅ El dataframe se ha creado correctamente
Shape: (10000, 20)

Columnas: ['listing_id', 'operation', 'district', 'neighborhood', 'address', 'surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'floor', 'elevator', 'balcony', 'furnished', 'condition', 'energy_certificate', 'has_parking', 'latitude', 'longitude', 'agency']

Primeras filas:


,listing_id,operation,district,neighborhood,address,surface_m2,rooms,bathrooms,price_eur,price_per_m2,floor,elevator,balcony,furnished,condition,energy_certificate,has_parking,latitude,longitude,agency
0,ID_0,alquiler,Unknown,Sagrada Família,C/ Aragó 395,89 m²,?,2,?,4240 €/m2,1º,Y,No,partially,average,?,No,NaN,?,Particular
1,NaN,VENDER,Eixampl,Les Corts,Passeig de Gràcia,171,NaN,1,?,7920.91,ático,?,N,NaN,?,D,No,NaN,?,Housfy
2,ID_2,lease,Sant Martí,El Clot,C/ Mallorca 316,?,2+,?,317642 €,?,2º,Y,?,?,average,D,?,41.3997,?,Engel & Völkers
3,NaN,alquiler,SANTS,Sagrada Família,Calle Falsa 123,NaN,three,two,NaN,5484 €/m2,sótano,N,Sí,?,a reformar,A,Y,NaN,2.0,Engel & Völkers
4,5,buy,SANTS,Les Corts,C/ Gran Via 245,?,2+,?,NaN,?,4º,Sí,N,Sí,average,F,Y,?,2.0,Particular


## TRANSFORM: Limpieza de Datos

### Paso 1: Crear copia para trabajar


In [17]:
# Crear copia del dataframe
df_clean = df_raw.copy()
print(f"✅ Dataframe copiado. Filas: {len(df_clean)}")


✅ Dataframe copiado. Filas: 10000


### Paso 2: Eliminar espacios (strip) en columnas de texto


In [18]:
# Aplicar strip() a todas las columnas de tipo object (string)
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col] = df_clean[col].astype(str).str.strip()
    # Reemplazar 'nan' string por NaN
    df_clean[col] = df_clean[col].replace('nan', np.nan)

print("✅ Espacios eliminados de todas las columnas de texto")


✅ Espacios eliminados de todas las columnas de texto


### Paso 3: Rellenar valores vacíos


In [19]:
# Reemplazar valores que representan "vacío" por NaN
valores_vacios = ['', ' ', 'nan', 'None', 'N/A', 'n/a', 'NULL', 'null', '?', 'unknown']

for col in df_clean.columns:
    df_clean[col] = df_clean[col].replace(valores_vacios, np.nan)

print("✅ Valores vacíos convertidos a NaN")
print(f"\nValores NaN por columna:")
print(df_clean.isnull().sum().sort_values(ascending=False))


✅ Valores vacíos convertidos a NaN

Valores NaN por columna:
latitude              5055
price_per_m2          5041
longitude             4919
rooms                 4004
bathrooms             3992
surface_m2            3983
furnished             3907
address               3904
price_eur             3347
listing_id            3321
balcony               2579
condition             2533
elevator              2451
has_parking           2447
energy_certificate    2234
neighborhood          1425
agency                1422
operation             1399
floor                 1277
district                 0
dtype: int64


### Paso 4: Convertir tipos de datos adecuados


In [20]:
# Función para extraer números de strings
def extract_number(value):
    """Extrae el primer número de un string"""
    if pd.isna(value):
        return np.nan
    value_str = str(value)
    # Buscar números (enteros o decimales)
    numbers = re.findall(r'\d+\.?\d*', value_str)
    if numbers:
        return float(numbers[0])
    return np.nan

# Función para convertir texto a número (one, two, three, etc.)
def text_to_number(value):
    """Convierte texto a número"""
    if pd.isna(value):
        return np.nan
    value_str = str(value).lower().strip()
    
    # Mapear texto a números
    text_map = {
        'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
        'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10
    }
    
    if value_str in text_map:
        return text_map[value_str]
    
    # Si tiene formato "2+", extraer el número
    if '+' in value_str:
        nums = re.findall(r'\d+', value_str)
        if nums:
            return int(nums[0])
    
    # Intentar extraer número directamente
    return extract_number(value)

# Limpiar surface_m2 (puede venir como "89 m²", "107m2", etc.)
if 'surface_m2' in df_clean.columns:
    df_clean['surface_m2'] = df_clean['surface_m2'].apply(extract_number)

# Limpiar rooms (puede venir como "three", "2+", etc.)
if 'rooms' in df_clean.columns:
    df_clean['rooms'] = df_clean['rooms'].apply(text_to_number)

# Limpiar bathrooms (similar a rooms)
if 'bathrooms' in df_clean.columns:
    df_clean['bathrooms'] = df_clean['bathrooms'].apply(text_to_number)

# Limpiar price_eur (puede venir como "317642 €", "1.200€", etc.)
if 'price_eur' in df_clean.columns:
    def extract_price(value):
        if pd.isna(value):
            return np.nan
        value_str = str(value).replace('€', '').replace('.', '').replace(',', '.').strip()
        numbers = re.findall(r'\d+\.?\d*', value_str)
        if numbers:
            return float(numbers[0])
        return np.nan
    df_clean['price_eur'] = df_clean['price_eur'].apply(extract_price)

# Limpiar price_per_m2 (puede venir como "4240 €/m2", etc.)
if 'price_per_m2' in df_clean.columns:
    def extract_price_m2(value):
        if pd.isna(value):
            return np.nan
        value_str = str(value).replace('€/m2', '').replace('€/m²', '').replace('.', '').replace(',', '.').strip()
        numbers = re.findall(r'\d+\.?\d*', value_str)
        if numbers:
            return float(numbers[0])
        return np.nan
    df_clean['price_per_m2'] = df_clean['price_per_m2'].apply(extract_price_m2)

# Convertir coordenadas
if 'latitude' in df_clean.columns:
    df_clean['latitude'] = pd.to_numeric(df_clean['latitude'], errors='coerce')
if 'longitude' in df_clean.columns:
    df_clean['longitude'] = pd.to_numeric(df_clean['longitude'], errors='coerce')

print("✅ Columnas numéricas limpiadas y convertidas")
print(f"\nTipos de datos numéricos:")
numeric_cols = ['surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'latitude', 'longitude']
for col in numeric_cols:
    if col in df_clean.columns:
        print(f"  {col}: {df_clean[col].dtype}")


✅ Columnas numéricas limpiadas y convertidas

Tipos de datos numéricos:
  surface_m2: float64
  rooms: float64
  bathrooms: float64
  price_eur: float64
  price_per_m2: float64
  latitude: float64
  longitude: float64


In [21]:
# Convertir columnas que deben ser enteros
int_cols = ['rooms', 'bathrooms']

for col in int_cols:
    if col in df_clean.columns:
        # Convertir a int, pero mantener NaN usando Int64
        df_clean[col] = df_clean[col].astype('Int64')  # Int64 permite NaN

print("✅ Columnas convertidas a enteros")


✅ Columnas convertidas a enteros


In [22]:
# Asegurar que las columnas de texto sean string
text_cols = ['listing_id', 'operation', 'district', 'neighborhood', 'address', 
             'floor', 'condition', 'energy_certificate', 'agency']

for col in text_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).replace('nan', np.nan)

print("✅ Columnas de texto convertidas a string")


✅ Columnas de texto convertidas a string


In [23]:
# Convertir columnas booleanas
boolean_cols = ['elevator', 'balcony', 'furnished', 'has_parking']

for col in boolean_cols:
    if col in df_clean.columns:
        # Normalizar valores booleanos
        df_clean[col] = df_clean[col].astype(str).str.lower().str.strip()
        df_clean[col] = df_clean[col].replace({
            'y': True, 'yes': True, 'sí': True, 'si': True, 's': True, '1': True, 'true': True, 'yes': True,
            'n': False, 'no': False, '0': False, 'false': False
        })
        # Reemplazar valores que no coincidan con NaN
        mask = ~df_clean[col].isin([True, False])
        df_clean.loc[mask, col] = np.nan

print("✅ Columnas booleanas convertidas")


✅ Columnas booleanas convertidas


### Paso 5: Rellenar valores faltantes


In [ ]:
# Rellenar valores faltantes
# Para columnas de texto (object): rellenar con "{nombre_columna} empty"
# Para columnas numéricas: rellenar con la media

print("=== RELLENANDO VALORES FALTANTES ===\n")

# Identificar columnas de texto (object) y numéricas
text_cols = df_clean.select_dtypes(include=['object']).columns
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns

# Rellenar columnas de texto
for col in text_cols:
    if df_clean[col].isnull().sum() > 0:
        fill_value = f"{col} empty"
        df_clean[col] = df_clean[col].fillna(fill_value)
        print(f"✅ {col}: valores rellenados con '{fill_value}'")

# Rellenar columnas numéricas con la media
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        mean_value = df_clean[col].mean()
        
        # Si es Int64, redondear la media a entero
        if df_clean[col].dtype == 'Int64':
            mean_value = int(round(mean_value))
            df_clean[col] = df_clean[col].fillna(mean_value)
            print(f"✅ {col}: valores rellenados con media = {mean_value} (entero)")
        else:
            df_clean[col] = df_clean[col].fillna(mean_value)
            print(f"✅ {col}: valores rellenados con media = {mean_value:.2f}")

print(f"\n✅ Todos los valores faltantes han sido rellenados")
print(f"Valores NaN restantes: {df_clean.isnull().sum().sum()}")


=== EJEMPLOS DE LIMPIEZA ===

ANTES (RAW):
  surface_m2  rooms bathrooms price_eur price_per_m2 elevator
0      89 m²      ?         2         ?    4240 €/m2        Y
1        171    NaN         1         ?      7920.91        ?
2          ?     2+         ?  317642 €            ?        Y
3        NaN  three       two       NaN    5484 €/m2        N
4          ?     2+         ?       NaN            ?       Sí
5     127 m²  three         2  491626 €          NaN        Y
6          ?     2+       two       NaN            ?        N
7          ?  three         ?   1282371    4093 €/m2        Y
8     127 m²     2+         3         ?       6630.1  unknown
9        NaN     2+       NaN      4512      7856.74       no

DESPUÉS (CLEAN):
   surface_m2  rooms  bathrooms  price_eur  price_per_m2 elevator
0        89.0   <NA>          2        NaN        4240.0     True
1       171.0   <NA>          1        NaN      792091.0      NaN
2         NaN      2       <NA>   317642.0           NaN   

### Verificación: Comparación antes/después


In [ ]:
# Mostrar ejemplos de limpieza
print("=== EJEMPLOS DE LIMPIEZA ===\n")
print("ANTES (RAW):")
print(df_raw[['surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'elevator', 'district']].head(10))
print("\nDESPUÉS (CLEAN):")
print(df_clean[['surface_m2', 'rooms', 'bathrooms', 'price_eur', 'price_per_m2', 'elevator', 'district']].head(10))


=== RESUMEN DE LA TRANSFORMACIÓN ===

Filas: 10000
Columnas: 20

Tipos de datos:
listing_id             object
operation              object
district               object
neighborhood           object
address                object
surface_m2            float64
rooms                   Int64
bathrooms               Int64
price_eur             float64
price_per_m2          float64
floor                  object
elevator               object
balcony                object
furnished              object
condition              object
energy_certificate     object
has_parking            object
latitude              float64
longitude             float64
agency                 object
dtype: object

Valores faltantes totales: 61291

Primeras filas del dataset limpio:


,listing_id,operation,district,neighborhood,address,surface_m2,rooms,bathrooms,price_eur,price_per_m2,floor,elevator,balcony,furnished,condition,energy_certificate,has_parking,latitude,longitude,agency
0,ID_0,alquiler,Unknown,Sagrada Família,C/ Aragó 395,89.0,<NA>,2,NaN,4240.0,1º,True,False,NaN,average,NaN,False,NaN,NaN,Particular
1,NaN,VENDER,Eixampl,Les Corts,Passeig de Gràcia,171.0,<NA>,1,NaN,792091.0,ático,NaN,False,NaN,NaN,D,False,NaN,NaN,Housfy
2,ID_2,lease,Sant Martí,El Clot,C/ Mallorca 316,NaN,2,<NA>,317642.0,NaN,2º,True,NaN,NaN,average,D,NaN,41.3997,NaN,Engel & Völkers
3,NaN,alquiler,SANTS,Sagrada Família,Calle Falsa 123,NaN,3,2,NaN,5484.0,sótano,False,True,NaN,a reformar,A,True,NaN,2.0,Engel & Völkers
4,5,buy,SANTS,Les Corts,C/ Gran Via 245,NaN,2,<NA>,NaN,NaN,4º,True,False,True,average,F,True,NaN,2.0,Particular


### Resumen de la transformación


In [ ]:
print("=== RESUMEN DE LA TRANSFORMACIÓN ===\n")
print(f"Filas: {len(df_clean)}")
print(f"Columnas: {len(df_clean.columns)}")
print(f"\nTipos de datos:")
print(df_clean.dtypes)
print(f"\nValores faltantes totales: {df_clean.isnull().sum().sum()}")
print(f"\nPrimeras filas del dataset limpio:")
df_clean.head()


## LOAD: Guardar datos limpios


In [ ]:
# Guardar el dataframe limpio
df_clean.to_csv("../data/housing-barcelona-clean.csv", index=False)

print("✅ Datos limpios guardados en: ../data/housing-barcelona-clean-pandas.csv")
print(f"\nArchivo guardado exitosamente con {len(df_clean)} filas y {len(df_clean.columns)} columnas")


✅ Datos limpios guardados en: ../data/housing-barcelona-clean.csv

Archivo guardado exitosamente con 10000 filas y 20 columnas
